In [ ]:
import numpy as np
import sklearn
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.preprocessing import normalize, MinMaxScaler
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
# Loading in all datafiles
X_train_scaled = np.load('X_train_scaled.npy')
X_test_scaled = np.load('X_test_scaled.npy')

X_train_original = np.load('X_train.npy')
y_train_original = np.load('y_train.npy')

# Data from SMOTE 
ns_SMOTE_X_train = np.load('ns_SMOTE_X_train.npy')
ns_SMOTE_y_train = np.load('ns_SMOTE_y_train.npy')

scholarship_SMOTE_X_train = np.load('scholarship_SMOTE_X_train.npy')
scholarship_SMOTE_y_train = np.load('scholarship_SMOTE_y_train.npy')

# Data from IHT
ns_IHT_X_train = np.load('ns_IHT_X_train.npy')
ns_IHT_y_train = np.load('ns_IHT_y_train.npy')

scholarship_IHT_X_train = np.load('scholarship_IHT_X_train.npy')
scholarship_IHT_y_train = np.load('scholarship_IHT_y_train.npy')

# Testing: (hasn't been changed)
X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')

# Raw: used for grouping 
X_test_raw = np.load('X_test.npy')

In [ ]:
# UPDATE: change based on which set of data you want to run it on
X_train = ns_SMOTE_X_train
y_train = ns_SMOTE_y_train

In [ ]:
# Standardization 
scaler = sklearn.preprocessing.StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Model Evaluation

In [ ]:
scholarship_index = np.where(X_test_raw[:, 2] == 1)

# Divide into separate testing sets 
y_test_scholarship = np.squeeze(np.take(y_test, scholarship_index))
y_test_noscholarship = np.delete(y_test, scholarship_index)
y_test_list = [y_test, y_test_scholarship, y_test_noscholarship]

In [ ]:
# Check that no data points were lost
print(len(y_test_scholarship) + len(y_test_noscholarship))
print(len(y_test))
print(len(y_test_scholarship))

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, precision_recall_curve, confusion_matrix, roc_curve, roc_auc_score, brier_score_loss, f1_score, auc
evaluation_stats = {}
ROC = []
PRCurve = []

# Helper function for displaying model evaluation metrics
def evaluate(y_pred, probs, model_name, print_toggle):
    # Overall ROC Curve 
    fpr, tpr, _ = roc_curve(y_test, probs)
    ROC.append({'label': model_name,
               'FPR': fpr,
               'TPR': tpr})
    
    # Precision-recall curve 
    p, r, _ = precision_recall_curve(y_test, probs)
    PRCurve.append({'label': model_name,
                   'recall': r, 
                   'precision': p})
    
    # split predictions into categories 
    y_pred_scholarship = np.squeeze(np.take(y_pred, scholarship_index))
    y_pred_noscholarship = np.delete(y_pred, scholarship_index)
    y_pred_list = [y_pred, y_pred_scholarship, y_pred_noscholarship]

    probs_scholarship = np.squeeze(np.take(probs, scholarship_index))
    probs_noscholarship = np.delete(probs, scholarship_index)
    probs_list = [probs, probs_scholarship, probs_noscholarship]
    
    # Organization of evaluation stats: 

    group_names = ["Overall", "Scholarship", "No Scholarship"]
    evaluation_stats[model_name] = {}
    
    fig, axes = plt.subplots(1, 3, figsize = (20, 5), sharey=True)
    for i in range(len(y_pred_list)):
        # convert from Boolean to 0 and 1 
        if (y_pred_list[i].dtype == bool):
            y_pred_list[i] = [1 if p == True else 0 for p in y_pred_list[i]]
        
        group = []
        
        # accuracy
        accuracy = accuracy_score(y_test_list[i], y_pred_list[i])
        group.append(accuracy)
        
        # recall
        recall = recall_score(y_test_list[i], y_pred_list[i])
        group.append(recall)
        
        # precision 
        precision = precision_score(y_test_list[i], y_pred_list[i])
        group.append(precision)
        
        # f1 score
        f1 = 2 * (precision * recall) / (precision + recall)
        group.append(f1)
        
        # false negative and false positive 
        tn, fp, fn, tp = confusion_matrix(y_test_list[i], y_pred_list[i]).ravel()
        fnr = fn / (fn + tp)
        group.append(fnr)
        fpr = fp / (fp + tn)
        group.append(fpr)
        
        # Group Confusion Matrix 
        cf_matrix = confusion_matrix(y_test_list[i], y_pred_list[i])

        l = ['True Neg','False Pos','False Neg','True Pos']
        l_counts = ["{0:0.0f}".format(value) for value in
                        cf_matrix.flatten()]
        l_percentages = ["{0:.2%}".format(value) for value in
                             cf_matrix.flatten()/np.sum(cf_matrix)]
        labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
                  zip(l,l_counts,l_percentages)]
        labels = np.asarray(labels).reshape(2,2)

        if(group_names[i] == "Overall"):
            sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues', ax = axes[i])
        else:
            sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='YlOrBr', ax = axes[i])
        axes[i].set_title(group_names[i] + " Confusion Matrix")
        axes[i].set_ylim([0,2])
        
        # AUC 
        auc = roc_auc_score(y_test_list[i], probs_list[i])
        group.append(auc)
        
        # Brier score 
        b_score = brier_score_loss(y_test_list[i], probs_list[i])
        group.append(b_score)
        
        evaluation_stats[model_name][group_names[i]] = group
        if(print_toggle):
            print(group_names[i])
            print("Accuracy: \t", accuracy)
            print("Recall: \t", recall)
            print("Precision: \t", precision)
            print("F1: \t", f1)
            print("False Negative Rate: \t", fnr)
            print("False Positive Rate: \t", fpr)
            print("AUC: \t", auc)
            print("Brier Score: \t", b_score)
            print()

## Feature Selection

In [ ]:
# for regression 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression 

selector = SelectKBest(f_regression, k = 20)
X_train_reg = selector.fit_transform(X_train, y_train)
cols = selector.get_support(indices = True)
print(cols)
X_test_reg = selector.fit_transform(X_test, y_test)

# for classification 
from sklearn.feature_selection import f_classif
selector = SelectKBest(f_classif, k = 20)
X_train_class = selector.fit_transform(X_train, y_train)
cols = selector.get_support(indices = True)
print(cols)
X_test_class = selector.fit_transform(X_test, y_test)

## Base Model
Predicting everyone is a show 

In [ ]:
y_pred = np.zeros(len(y_test))
y_cal = np.zeros(len(y_test))
evaluate(y_pred, y_cal, "Base", True)

## OLS

In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
distributions = dict(fit_intercept=[True], normalize=[True])
search = GridSearchCV(reg, distributions).fit(X_train_reg, y_train)
print(search.best_params_)

y_cal = search.predict(X_test_reg)
scaler = MinMaxScaler()
y_cal = scaler.fit_transform(y_cal.reshape(-1, 1))

# due to Python rounding error
for i in range(len(y_cal)):
    if y_cal[i] > 1:
        y_cal[i] = 1

y_pred = y_cal > 0.5
accuracy_score(y_test, y_pred)

In [ ]:
evaluate(y_pred, y_cal, "OLS", True)

## Elastic Net

In [ ]:
reg = linear_model.ElasticNet()

grid = dict()
grid['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
grid['l1_ratio'] = np.arange(0, 1, 0.01)
search = RandomizedSearchCV(reg, grid).fit(X_train_reg, y_train)
print(search.best_params_)

y_cal = search.predict(X_test_reg)
scaler = MinMaxScaler()
y_cal = scaler.fit_transform(y_cal.reshape(-1, 1))


# due to Python rounding error
for i in range(len(y_cal)):
    if y_cal[i] > 1:
        y_cal[i] = 1
        
y_pred = y_cal > 0.5
accuracy_score(y_test, y_pred)

In [ ]:
evaluate(y_pred, y_cal, "Elastic Net", True)

## Logistic

In [ ]:
clf = linear_model.LogisticRegression()
grid = dict()
grid['C'] = np.arange(0, 100, 0.01)
search = RandomizedSearchCV(clf, grid).fit(X_train_reg, y_train)

print(search.best_params_)

y_pred = search.predict(X_test_reg)
accuracy_score(y_test, y_pred)

In [ ]:
# uncalibrated 
y_uncal = search.predict_proba(X_test_reg)[:, 1]
# calibrated 
calibrator = CalibratedClassifierCV(search, cv = 3)
calibrator.fit(X_test_reg, y_test)
y_cal = calibrator.predict_proba(X_test_reg)[:, 1]

In [ ]:
evaluate(y_pred, y_cal, "Logistic", True)

## SVM

In [ ]:
from sklearn.svm import SVC
svm = SVC(probability = True, kernel = 'rbf', max_iter = 10)
grid = dict()
grid['gamma'] = np.arange(0, 10, 0.01)
grid['C'] = np.arange(0, 100, 0.01)
#grid['max_iter'] = [20]

search = RandomizedSearchCV(svm, grid).fit(X_train_class, y_train)
print(search.best_params_)

y_pred = search.predict(X_test_class)
accuracy_score(y_test, y_pred)

In [ ]:
# uncalibrated 
y_uncal = search.predict_proba(X_test_class)[:, 1]
# calibrated 
calibrator = CalibratedClassifierCV(search, cv = 3)
calibrator.fit(X_test_class, y_test)
y_cal = calibrator.predict_proba(X_test_class)[:, 1]

In [ ]:
evaluate(y_pred, y_cal, "SVM", True)

## Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors = 8, p = 1)
#grid = dict()
#grid['n_neighbors'] = np.arange(1, 25, 1)
#grid['p'] = [1, 2, 3]

search = neigh.fit(X_train_class, y_train)
#print(search.best_params_)

y_pred = search.predict(X_test_class)
accuracy_score(y_test, y_pred)

In [ ]:
# uncalibrated 
y_uncal = search.predict_proba(X_test_class)
# calibrated 
calibrator = CalibratedClassifierCV(search, cv = 3)
calibrator.fit(X_test_class, y_test)
y_cal = calibrator.predict_proba(X_test_class)[:, 1]

In [ ]:
evaluate(y_pred, y_cal, "kNN", True)

## Decision Trees

In [ ]:
from sklearn import tree
dt = tree.DecisionTreeClassifier()
grid = dict()
grid['criterion'] = ["gini"]
grid['max_depth'] = np.arange(5, 10, 1)
grid['min_samples_leaf'] = np.arange(2, 10, 1)
grid['min_samples_split'] = np.arange(2, 10, 1)

search = RandomizedSearchCV(dt, grid).fit(X_train_class, y_train)
print(search.best_params_)

y_pred = search.predict(X_test_class)
accuracy_score(y_test, y_pred)

In [ ]:
# uncalibrated 
y_uncal = search.predict_proba(X_test_class)
# calibrated 
calibrator = CalibratedClassifierCV(search, cv = 3)
calibrator.fit(X_test_class, y_test)
y_cal = calibrator.predict_proba(X_test_class)[:, 1]

In [ ]:
evaluate(y_pred, y_cal, "Decision Trees", True)

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_features = "auto", criterion = "entropy")
grid = dict()
grid['n_estimators'] = np.arange(2, 20, 2)
grid['max_depth'] = np.arange(5, 50, 5)
grid['min_samples_leaf'] = np.arange(2, 10, 1)
grid['min_samples_split'] = np.arange(2, 10, 1)

search = RandomizedSearchCV(rf, grid).fit(X_train_class, y_train)
print(search.best_params_)

y_pred = search.predict(X_test_class)
accuracy_score(y_test, y_pred)

In [ ]:
# uncalibrated 
y_uncal = search.predict_proba(X_test_class)
# calibrated 
calibrator = CalibratedClassifierCV(search, cv = 3)
calibrator.fit(X_test_class, y_test)
y_cal = calibrator.predict_proba(X_test_class)[:, 1]

In [ ]:
evaluate(y_pred, y_cal, "Random Forest", True)

## Neural Network Models

In [ ]:
from sklearn.neural_network import MLPClassifier

nn = MLPClassifier(activation = "logistic", early_stopping = True, validation_fraction = 0.1, learning_rate = "invscaling", max_iter = 100)
grid = dict()
grid['hidden_layer_sizes'] = [(25, ), (50, ), (75, ), (100, )]
grid['alpha'] = np.arange(0, 10, 0.01)
grid['batch_size'] = np.arange(6, 512, 8)

search = RandomizedSearchCV(nn, grid).fit(X_train_class, y_train)
print(search.best_params_)

y_pred = search.predict(X_test_class)
accuracy_score(y_test, y_pred)

In [ ]:
# uncalibrated 
y_uncal = search.predict_proba(X_test_class)
# calibrated 
calibrator = CalibratedClassifierCV(search, cv = 3)
calibrator.fit(X_test_class, y_test)
y_cal = calibrator.predict_proba(X_test_class)[:, 1]

In [ ]:
evaluate(y_pred, y_cal, "MLP", True)

## Export Evaluation Metrics to Text File

In [ ]:
import pprint

pprint.pprint(evaluation_stats)

In [ ]:
f = open("ns_SMOTE_evaluation_stats.txt","w")
f.write( str(evaluation_stats) )
f.close()

## Combined ROC and PR Curves

In [ ]:
for x in ROC:
    plt.plot(x['FPR'], x['TPR'], label = x['label'])
plt.title("ROC Curve")
plt.xlabel("Specificity (FPR)")
plt.ylabel("Sensitivity (TPR)")
plt.legend(loc = "lower right")
plt.show()

In [ ]:
for x in PRCurve:
    plt.plot(x['recall'], x['precision'], label = x['label'])
plt.title("Precision-Recall Curve")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.legend(loc = "lower right")
plt.show()